In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

from scipy import optimize, stats

from numpy import sqrt, cos, sin


import numiweight

In [ ]:
plt.rcParams.update({'font.size': 16})
# mpl.rcParams['lines.linewidth'] = 4.

dosave = True
savedir = "plots_2_28_24/kaonstudy/"

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
Masses = np.array([220, 240, 260, 280, 300, 320, 340])

fs = ["/icarus/data/users/gputnam/DMCP2023G/mc-F-iter2/F2-Higgs_M%i_nom_evt.df" % M for M in Masses]

In [ ]:
dfs = [pd.read_hdf(f, "mcnu") for f in fs]
for i in range(len(dfs)):
    dfs[i].index = dfs[i].index.droplevel(-1)

In [ ]:
dfhs = [pd.read_hdf(f, "mch") for f in fs]
for i in range(len(dfhs)):
    dfhs[i].index = dfhs[i].index.droplevel(-1)

In [ ]:
# get weights

for i in range(len(dfs)):
    nupdg = dfs[i].pdg
    nuparent_pdg = dfs[i].parent_pdg
    nuE = dfs[i].E
    
    ppfx_cv = numiweight.cv(nupdg, nuE)
    concrete_cv = numiweight.concrete_cv(nuparent_pdg, nuE)
    
    dfs[i]["weight"] = ppfx_cv * concrete_cv

In [ ]:
for iplt in range(len(Masses)):
    plt.figure(iplt)
    var = dfhs[iplt].E
    weights = dfs[iplt].weight

    klong = dfs[iplt].parent_pdg == 130
    kplus = np.abs(dfs[iplt].parent_pdg) == 321

    _ = plt.hist(var[klong], weights=weights[klong], bins=np.linspace(0, 4, 11), histtype="step", density=True, label="$K^L$ Production", linewidth=2)
    _ = plt.hist(var[kplus], weights=weights[kplus], bins=np.linspace(0, 4, 11), histtype="step", density=True, label="$K^\\pm$ Production", linewidth=2)
    plt.ylabel("Area Normalized")
    plt.xlabel("Scalar Energy [GeV]")
    plt.legend(title="M: %iMeV, $\\theta_S$: %.0e" % (Masses[iplt], dfhs[iplt].C1.iloc[0]))
    plt.tight_layout()
    
    if dosave:
        plt.savefig(savedir + "kaon_relative_spectra_M%i.pdf" % int(Masses[iplt]))
        plt.savefig(savedir + "kaon_relative_spectra_M%i.svg" % int(Masses[iplt]))
        plt.savefig(savedir + "kaon_relative_spectra_M%i.png" % int(Masses[iplt]))

In [ ]:
fracs = np.array([((df.parent_pdg == 130)).sum() / ((np.abs(df.parent_pdg) == 321)).sum() for df in dfs])

In [ ]:
plt.plot(Masses, fracs)

In [ ]:
fracs = np.array([(df.weight*(df.parent_pdg == 130)).sum() / (df.weight*(np.abs(df.parent_pdg) == 321)).sum() for df in dfs])

In [ ]:
plt.plot(Masses, fracs)

In [ ]:
# CONSTANTS

M_PI = np.pi
elec_mass = 0.0005109989461
muon_mass = 0.1056583745
tau_mass  = 1.77686
piplus_mass = 0.13957039
pizero_mass = 0.1349768
kplus_mass = 0.493677
klong_mass = 0.497611
tquark_mass = 172.76
eta_mass = 0.547862
rho_mass = 0.77526
etap_mass = 0.95778
Wmass = 80.377

fine_structure_constant = 7.2973525693e-3
Gfermi = 1.166379e-5
higgs_vev = 1. / sqrt(sqrt(2)*Gfermi)
sin2thetaW = 0.2312
gL = -0.5 + sin2thetaW
gR = sin2thetaW

fpion = 0.1302 / sqrt(2) # 93 MeV convention

# compute the eta decay constants
# From https://link.springer.com/article/10.1140/epjc/s10052-021-08861-y
f0 = 0.148
f8 = 0.165
th0 = -6.9*M_PI/180 # rad
th8 = -21.2*M_PI/180. # rad
feta = cos(th8)*f8/sqrt(3) + sin(th0)*f0/sqrt(6) #eq 83
fetap = sin(th8)*f8/sqrt(3) - cos(th0)*f0/sqrt(6) #eq 83

frho = 0.171# GeV^2
grho = 1 - 2*sin2thetaW # // https://link.springer.com/article/10.1140/epjc/s10052-021-08861-y

# unit conversion
hbar = 6.582119569e-16 # GeV*ns or eV*s https://pdg.lbl.gov/2020/reviews/rpp2020-rev-phys-constants.pdf
c_cm_per_ns = 29.9792458 # cm / ns https://pdg.lbl.gov/2020/reviews/rpp2020-rev-phys-constants.pdf

# kaon lifetimes
kplus_lifetime = 1.238e1 # ns https://pdg.lbl.gov/2020/listings/rpp2020-list-K-plus-minus.pdf
klong_lifetime = 5.116e1 # ns https://pdg.lbl.gov/2020/listings/rpp2020-list-K-zero-L.pdf (FIT)
kshort_lifetime = 8.954e-2 # ns https://pdg.lbl.gov/2014/tables/rpp2014-tab-mesons-strange.pdf


# other lifetimes
tau_lifetime = 290.3e-6 # ns https://pdg.lbl.gov/2019/tables/rpp2019-sum-leptons.pdf

# and widths
rho_width = 0.1478 # GeV https://pdg.lbl.gov/2019/listings/rpp2019-list-rho-770.pdf

# Kaon decay branching ratios
kaonp_mup_numu = 0.6356 # From PDG: https://pdg.lbl.gov/2020/listings/rpp2020-list-K-plus-minus.pdf
kaonp_ep_nue = 1.582e-5 # From PDG: https://pdg.lbl.gov/2020/listings/rpp2020-list-K-plus-minus.pdf
kshort_2pi = 0.692 # https://pdg.lbl.gov/2014/tables/rpp2014-tab-mesons-strange.pdf

# CKM matrix
abs_Vud_squared = 0.97370 * 0.97370 # https://pdg.lbl.gov/2020/reviews/rpp2020-rev-ckm-matrix.pdf (12.7)

abs_VtsVtd_squared = 1.19777e-7
rel_VtsVtd_squared = 1.02136e-7

In [ ]:
def child_momentum(M, m1, m2) :
    return sqrt(M * M * M * M
        -2 * M * M * m1 * m1
        -2 * M * M * m2 * m2
           + m1 * m1 * m1 * m1
           + m2 * m2 * m2 * m2
        -2 * m1 * m1 * m2 * m2) / ( 2 * M )

In [ ]:
higgs_momentum = child_momentum

def HPS_KaonPlusBranchingRatio(higs_mass, mixing):
    # kplus width
    #
    # matrix eement for kplus
    M_KP = (1. / 2.) * ( 3. / (16. * M_PI * M_PI * higgs_vev * higgs_vev * higgs_vev)) * (kplus_mass * kplus_mass) * (tquark_mass * tquark_mass) * mixing
    M_KP2 = M_KP * M_KP * abs_VtsVtd_squared

    kplus_width = (2 * higgs_momentum(kplus_mass, piplus_mass, higs_mass)/kplus_mass) * M_KP2 / (16 * M_PI * kplus_mass)

    # convert to partia ifetime
    kplus_2higgs_lifetime = hbar / kplus_width 

    # and branching ratio
    #
    # (this higgs decay woud make a negigibe contribution to the overa ifetime)
    return kplus_lifetime / kplus_2higgs_lifetime
        

def HPS_KaonLongBranchingRatio(higs_mass, mixing):
    M_KL = (1. / 2.) * (3. / (16. * M_PI * M_PI * higgs_vev * higgs_vev * higgs_vev)) * (klong_mass * klong_mass) * (tquark_mass * tquark_mass) * mixing
    M_KL2 = M_KL * M_KL * rel_VtsVtd_squared

    klong_width = (2 * higgs_momentum(klong_mass, pizero_mass, higs_mass) / klong_mass) * M_KL2 / (16 * M_PI * klong_mass)

    klong_2higgs_lifetime = hbar / klong_width

    return klong_lifetime / klong_2higgs_lifetime


# Get the partial width for lepton decays
def LeptonPartialWidth(lep_mass, higs_mass, mixing):
    width = (mixing * mixing * lep_mass * lep_mass * higs_mass / (8 * M_PI * higgs_vev * higgs_vev)) * pow(1 - 4 * lep_mass * lep_mass / (higs_mass * higs_mass), 3. / 2.)
    width[(lep_mass * 2 >= higs_mass)] = 0
    return width


def HPS_ElectronPartialWidth(higs_mass, mixing):
    return LeptonPartialWidth(elec_mass, higs_mass, mixing)


def HPS_MuonPartialWidth(higs_mass, mixing):
    return LeptonPartialWidth(muon_mass, higs_mass, mixing)


def PionPartialWidth(pion_mass, higs_mass, mixing):
    form_factor = (2. / 9.) * higs_mass * higs_mass + (11. / 9.) * pion_mass * pion_mass

    width = (mixing * mixing * 3 * form_factor * form_factor / (32 * M_PI * higgs_vev * higgs_vev * higs_mass)) * pow(1- 4. * pion_mass * pion_mass / (higs_mass * higs_mass), 1. / 2.)

    width[(pion_mass * 2 >= higs_mass)] = 0
    return width


def HPS_PiPlusPartialWidth(higs_mass, mixing):
    return PionPartialWidth(piplus_mass, higs_mass, mixing)


def HPS_PiZeroPartialWidth(higs_mass, mixing):
    return PionPartialWidth(pizero_mass, higs_mass, mixing)

    

In [ ]:
def HPS_KBR(higs_mass, mixing, klong_frac=0.13):
    return HPS_KaonPlusBranchingRatio(higs_mass, mixing) + HPS_KaonLongBranchingRatio(higs_mass, mixing)*klong_frac

In [ ]:
BR_frac = HPS_KaonPlusBranchingRatio(Masses/1e3, 1e-5) / HPS_KaonLongBranchingRatio(Masses/1e3, 1e-5)

In [ ]:
plt.plot(Masses, fracs*BR_frac, linewidth=2)
plt.xlabel("Scalar Mass [MeV]")
plt.ylabel("Ratio of $K^L$ to $K^+$ Parentage,\nDe-weighting HPS BR")
plt.tight_layout()


plt.ylim([0.11, 0.13])

if dosave:
    plt.savefig(savedir + "klkp_rate_ratio.pdf")
    plt.savefig(savedir + "klkp_rate_ratio.svg")
    plt.savefig(savedir + "klkp_rate_ratio.png")